# 6. 영상 필터

## 6.1 컨볼루션과 블러링

* dst = cv2.filter2D(src, ddepth, kernel, dst, anchor, delta, borderType)
    * src: 입력 영상, Numpy 배열
    * ddepth: 출력 영상의 dtype (-1: 입력 영상과 동일)
        * -1: 입력 영상과 동일 (책에서는 주로 이 값으로 함)
        * CV_8U, CV_16U, CV_16S, CV_32F, CV_64F
    * kernel: 컨볼루션 커널, float32의 n x n 크기 배열, 홀수
        * 얘가 마스크!!
    * dst(optional): 결과 영상
    * anchor(optional): 커널의 기준점, default: 중심점 (-1, -1)
        * 특별하게 안줘도 된다.
    * delta(optional): 필터가 적용된 결과에 추가할 값
    * borderType(optional): 외곽 픽셀 보정 방법 지정

### 6.1.2 평균 블러링


In [20]:
import cv2
import numpy as np

img = cv2.imread('img/girl.jpg')

kernel = np.ones((5,5)) / 5**2
blured = cv2.filter2D(img, -1, kernel)

cv2.imshow('origin', img)
cv2.imshow('avrg blur', blured)
cv2.waitKey()
cv2.destroyAllWindows()


* dst = cv2.blur(src, ksize, dst, anchor, borderType)
    * src: 입력 영상, numpy 배열
    * ksize: 커널의 크기
    * 나머지 파라미터는 cv2.filter2D()와 동일

* dst = cv2.boxFilter(src, ddepth, ksize, dst, anchor, normalize, borderType)
    * src: 입력 영상
    * ddepth: 출력 영상의 dtype (-1: 입력 영상과 동일)
    * normalize(optional): 커널 크기로 정규화(1/ksize²) 지정 여부 (Boolean),   
    * default=True
    * 나머지 파라미터는 cv2.filter2D()와 동일

In [2]:
# 블러 전용 함수로 블러링 적용 (blur_avg_api.py)

import cv2
import numpy as np

file_name = 'img/taekwonv1.jpg'
img = cv2.imread(file_name)

# blur() 함수로 블러링  ---①
blur1 = cv2.blur(img, (10,10))
# boxFilter() 함수로 블러링 적용 ---②
blur2 = cv2.boxFilter(img, -1, (10,10))

# 결과 출력
merged = np.hstack( (img, blur1, blur2))
cv2.imshow('blur', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 6.1.3 가우시안 블러링
* cv2.GaussianBlur(src, ksize, sigmaX, sigmaY, borderType)
    * src: 입력 영상
    * ksize: 커널 크기 (주로 홀수)
    * sigmaX: X 방향 표준편차 (0: auto)
        * 0: auto, sigma = 0.3(ksize-1)(0.5-1)+0.8
    * sigmaY(optional): Y 방향 표준편차 (default: sigmaX)
        * defaule: sigmaX
    * borderType(optional): 외곽 테두리 보정 방식

* ret = cv2.getGaussianKernel(ksize, sigma, ktype)
* ret: 가우시안 커널 (1차원이므로 ret * ret.T 형태로 사용해야 함)


In [21]:
import cv2
import numpy as np

img = cv2.imread('img/gaussian_noise.jpg')

k1 = np.array([[1,2,1],[2,4,2],[1,2,1]]) * (1/16)
blur1 = cv2.filter2D(img, -1, k1)

k2 = cv2.getGaussianKernel(3, 0)
blur2 = cv2.filter2D(img, -1, k2*k2.T)

blur3 = cv2.GaussianBlur(img, (3,3), 0)

print('k1:', k1)
print('k2:', k2*k2.T)

merged = np.hstack((img, blur1, blur2, blur3))
cv2.imshow('gaussian blur', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

k1: [[0.0625 0.125  0.0625]
 [0.125  0.25   0.125 ]
 [0.0625 0.125  0.0625]]
k2: [[0.0625 0.125  0.0625]
 [0.125  0.25   0.125 ]
 [0.0625 0.125  0.0625]]


### 6.1.4 미디언 블러링

* dst = cv2.medianBlur(src, ksize)
    * src: 입력 영상
    * ksize: 커널 크기

In [16]:
import cv2
import numpy as np

img = cv2.imread('img/morph_dot.png')

blur = cv2.medianBlur(img, 5)

# 결과 출력 
merged = np.hstack((img,blur))
cv2.imshow('media', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 6.1.5 바이레터럴(bilateral) 필터

* dst = cv2.bilateralFilter(src, d, sigmaColor, sigmaSpace, dst, borderType)
    * src: 입력 영상
    * d: 필터의 직경(diameter), 5보다 크면 매우 느림
    * sigmaColor: 색공간의 시그마 값
    * sigmaSpace: 좌표 공간의 시그마 값(단순한 사용을 위해 sigmaColor와 sigmaSpace에 같은 값을 사용할 것을 권장, 범위는 10~150 권장)

In [18]:
# 바이레터럴 필터와 가우시안 필터 비교 (blur_bilateral.py)

import cv2
import numpy as np

img = cv2.imread("img/gaussian_noise.jpg")

# 가우시안 필터 적용 ---①
blur1 = cv2.GaussianBlur(img, (5,5), 0)

# 바이레터럴 필터 적용 ---②
blur2 = cv2.bilateralFilter(img, 5, 130, 130)

# 결과 출력
merged = np.hstack((img, blur1, blur2))
cv2.imshow('bilateral', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 6.2 경계 검출
* 배경과 전경을 분리하기 위한 기본 작업
* 경계를 흐릿하게 만드는 작업 --> 블러링(blurring)
* 경계를 선명하게 만드는 작업 --> 샤프닝(sharpening)
    * 경계를 검출한 다음 경계에 있는 픽셀만 강조한 것

### 6.2.1 기본 미분 필터
* 경계 검출: 픽셀 값의 변화가 갑자기 크게 일어나는 지점을 찾아내는 것 --> 미분 연산
* 영상(2차원)의 미분 --> x,y축 방향의 편미분 이용
    * x축과 y축 방향에서 다음 픽셀값에서 현재 픽셀값을 뺌.
    * x방향 미분 마스크 --> 세로방향 경계 검출
    * y방향 미분 마스크 --> 가로방향 경계 검출
* 경계의 강도(크기)와 방향
    * 강도(magnitude): sqrt(Gx\*2 + Gy\*2)
    * 방향(direction): arctan(Gx/Gy)

In [22]:
# 미분 커널로 경계 검출 (edge_differential.py)

import cv2
import numpy as np

img = cv2.imread("img/sudoku.jpg")

#미분 커널 생성 ---①
gx_kernel = np.array([[ -1, 1]])
gy_kernel = np.array([[ -1],[ 1]])

# 필터 적용 ---②
edge_gx = cv2.filter2D(img, -1, gx_kernel)
edge_gy = cv2.filter2D(img, -1, gy_kernel)
# 결과 출력
merged = np.hstack((img, edge_gx, edge_gy))
cv2.imshow('edge', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [25]:
# 로버츠 교차 필터를 적용한 경계 검출 (edge_roberts.py)

import cv2
import numpy as np

img = cv2.imread("img/sudoku.jpg")

# 로버츠 커널 생성 ---①
gx_kernel = np.array([[1,0], [0,-1]])
gy_kernel = np.array([[0, 1],[-1,0]])

# 커널 적용 ---② 
edge_gx = cv2.filter2D(img, -1, gx_kernel)
edge_gy = cv2.filter2D(img, -1, gy_kernel)

# 결과 출력
merged = np.hstack((img, edge_gx, edge_gy, edge_gx+edge_gy))
cv2.imshow('roberts cross', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
# 프리윗 마스크를 적용한 경계 검출 (edge_prewitt.py)

import cv2
import numpy as np

file_name = "img/sudoku.jpg"
img = cv2.imread(file_name)

# 프리윗 커널 생성
gx_k = np.array([[-1,0,1], [-1,0,1],[-1,0,1]])
gy_k = np.array([[-1,-1,-1],[0,0,0], [1,1,1]])

# 프리윗 커널 필터 적용
edge_gx = cv2.filter2D(img, -1, gx_k)
edge_gy = cv2.filter2D(img, -1, gy_k)

# 결과 출력
merged = np.hstack((img, edge_gx, edge_gy, edge_gx+edge_gy))
cv2.imshow('prewitt', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 6.2.4 소벨(Sobel) 필터


* dst = cv2.Sobel(src, ddepth, dx, dy, dst, ksize, scale, delta, borderType)
    * src: 입력 영상
    * ddepth: 출력 영상의 dtype (-1: 입력 영상과 동일)
    * dx, dy: 미분 차수 (0, 1, 2 중 선택, 둘 다 0일 수는 없음)
    * ksize: 커널의 크기 (1, 3, 5, 7 중 선택)
    * scale: 미분에 사용할 계수
    * delta: 연산 결과에 가산할 값

In [26]:
# 소벨 마스크를 적용한 경계 검출 (edge_sobel.py)

import cv2
import numpy as np

img = cv2.imread("img/sudoku.jpg")

# 소벨 커널을 직접 생성해서 엣지 검출 ---①
## 소벨 커널 생성
gx_k = np.array([[-1,0,1], [-2,0,2],[-1,0,1]])
gy_k = np.array([[-1,-2,-1],[0,0,0], [1,2,1]])
## 소벨 필터 적용
edge_gx = cv2.filter2D(img, -1, gx_k)
edge_gy = cv2.filter2D(img, -1, gy_k)

# 소벨 API를 생성해서 엣지 검출
sobelx = cv2.Sobel(img, -1, 1, 0, ksize=3)
sobely = cv2.Sobel(img, -1, 0, 1, ksize=3) 

# 결과 출력
merged1 = np.hstack((img, edge_gx, edge_gy, edge_gx+edge_gy))
merged2 = np.hstack((img, sobelx, sobely, sobelx+sobely))
merged = np.vstack((merged1, merged2))
cv2.imshow('sobel', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 6.2.5 샤르(Scharr) 필터

* dst = cv2.Scharr(src, ddepth, dx, dy, dst, scale, delta, borderType)
    * ksize가 없다는 것을 제외하면 모든 파라미터는 cv2.Sobel()과 동일합니다.

In [27]:
# 샤르 마스크를 적용한 경계 검출 (edge_scharr.py)

import cv2
import numpy as np

img = cv2.imread("img/sudoku.jpg")

# 샤르 커널을 직접 생성해서 엣지 검출 ---①
gx_k = np.array([[-3,0,3], [-10,0,10],[-3,0,3]])
gy_k = np.array([[-3,-10,-3],[0,0,0], [3,10,3]])
edge_gx = cv2.filter2D(img, -1, gx_k)
edge_gy = cv2.filter2D(img, -1, gy_k)

# 샤르 API로 엣지 검출 ---②
scharrx = cv2.Scharr(img, -1, 1, 0)
scharry = cv2.Scharr(img, -1, 0, 1)

# 결과 출력
merged1 = np.hstack((img, edge_gx, edge_gy))
merged2 = np.hstack((img, scharrx, scharry))
merged = np.vstack((merged1, merged2))
cv2.imshow('Scharr', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 6.2.6 라플라시안 필터 (Laplacian Filter)
* 라플라시안 필터는 2차 미분을 적용한 필터입니다. 경계를 더 제대로 검출할 수 있습니다.

*  dst = cv2.Laplacian(src, ddepth, dst, ksize, scale, delta, borderType)
    * 파라미터는 cv2.Sobel()과 동일합니다.

In [28]:
# 라플라시안 마스크를 적용한 경계 검출 (edge_laplacian.py)

import cv2
import numpy as np

img = cv2.imread("img/sudoku.jpg")

# 라플라시안 필터 적용 ---①
edge = cv2.Laplacian(img, -1)

# 결과 출력
merged = np.hstack((img, edge))
cv2.imshow('Laplacian', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 6.2.7 캐니 엣지 (Canny Edge)
* 캐니 엣지는 지금까지 살펴본 것처럼 한 가지 필터만 사용하는 것이 아니라 다음의 4단계 알고리즘에 따라 경계를 검출합니다. 

    1. 노이즈 제거: 5 x 5 가우시안 블러링 필터로 노이즈 제거
    2. 경계 그레디언트 방향 계산: 소벨 필터로 경계 및 그레디언트 방향 검출
    3. 비최대치 억제(Non-Maximum Suppression): 그레디언트 방향에서 검출된 경계 중 가장 큰 값만 선택하고 나머지는 제거
    4. 이력 스레시홀딩: 두 개의 경계 값(Max, Min)을 지정해서 경계 영역에 있는 픽셀들 중 큰 경계 값(Max) 밖의 픽셀과 연결성이 없는 픽셀 제거

* edges = cv2.Canny(img, threshold1, threshold2, edges, apertureSize, L2gardient)
    * img: 입력 영상
    * threshold1, threshold2: 이력 스레시홀딩에 사용할 Min, Max 값
    * apertureSize: 소벨 마스크에 사용할 커널 크기
    * L2gradient: 그레디언트 강도를 구할 방식 (True: 제곱 합의 루트 False: 절댓값의 합)
        * True:
        * False: 더 큰값
    * edges: 엣지 결과 값을 갖는 2차원 배열

In [30]:
# 캐니 엣지 검출 (edge_canny.py)

import cv2, time
import numpy as np

img = cv2.imread("img/sudoku.jpg")

# 케니 엣지 적용 
edges = cv2.Canny(img,100,200)

# 결과 출력
cv2.imshow('Original', img)
cv2.imshow('Canny', edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 6.3 모폴로지(Mopology)

* 모폴로지: 노이즈 제거, 구멍 메꾸기, 연결되지 않은 경계 이어붙이기 등 형태학적 관점에서의 영상 연산
    * 침식: 깎아내기
    * 팽창: 영역 확장

### 6.3.1 침식연산
* 구조화 요소
    * 0, 1로 채워진 커널
    * 1로 채워진 모양에 따라 사각, 타원, 십자 등이 있음
* 침식 연산
    * 구조화 요소 커널을 입력 영상에 적용해 1로 채워진 영역을 온전히 올려 놓을 수 없으면 해당 픽셀을 0으로 변경
    * 아주 작은 노이즈를 제거하거나 원래 따로 떨어진 물체인데 겹쳐져 하나으 ㅣ물체로 보일 때 서로 떼어내는데 효과적

* cv2.getStructuringElement(shape, ksize, anchor)
    * shape: 구조화 요소 커널 모양
        * cv2.MORPH_RECT: 사각형
        * cv2.MORPH_ELLIPSE: 타원형
        * cv2.MORPH_CROSS: 십자형
    * ksize: 커널 크기
    * anchor(optional): 구조화 요소의 기준점, cv2.MORPH_CROSS에만 의미 있으며 기본 값은 중심점 (-1, -1)

* dst = cv2.erode(src, kernel, anchor, iterations, borderType, borderValue)
    * src: 입력 영상, 바이너리
    * kernel: 구조화 요소 커널
    * anchor(optional): cv2.getStructuringElement()와 동일
    * iterations(optional): 침식 연산 적용 반복 횟수 중요!!! 이거 쓸듯
    * boderType(optional): 외곽 영역 보정 방법 
    * boderValue(optional): 외곽 영역 보정 값

In [16]:
# 침식 연산 (morph_erode.py)

import cv2
import numpy as np

img = cv2.imread('img/morph_dot.png')

# 구조화 요소 커널, 사각형 (3x3) 생성 ---①
k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
# 침식 연산 적용 ---②
erosion = cv2.erode(img, k, iterations=2)
erosion = cv2.dilate(erosion, k, iterations=2)

# 결과 출력
merged = np.hstack((img, erosion))
cv2.imshow('Erode', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 6.3.2 팽창 연산
* 팽창 연산
    * 구조화 요소 커널을 입력 영상에 적용해서 1로 채워진 영역이 한 픽셀이라도 덮이면 해당 픽셀을 1로 변경
    * 물체에 생긴 작은 구멍을 메꾸거나, 원래는 붙어있는 물체인데 분리되어있으면 하나로 붙이는 데 효과적

* dst = cv2.dilate(src, kernel, dst, anchor, iterations, bordeType, borderValue)
    * 모든 파라미터는 cv2.erode()와 동일합니다.

In [51]:
# 팽창 연산 (morph_dilate.py)

import cv2
import numpy as np

img = cv2.imread('img/morph_hole.png')

# 구조화 요소 커널, 사각형 (3x3) 생성 ---①
k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
# 팽창 연산 적용 ---②
dst = cv2.dilate(img, k, iterations=3)

# 결과 출력
merged = np.hstack((img, dst))
cv2.imshow('Dilation', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 6.3.2 팽창 연산
* 침식은 어두운 부분의 노이즈를 제거하는 효과가 있고 팽창은 밝은 부분의 노이즈를 제거하는 효과가 있다는 것을 살펴봤습니다. 노이즈 제거 효과는 좋으나 원래 모양이 홀쭉해지거나 뚱뚱해지는 변형이 일어납니다. 하지만 침식과 팽창의 연산을 조합하면 원래의 모양을 유지하면서 노이즈를 제거하는 효과를 거둘 수 있습니다.

* 침식 연산 후 팽창 연산을 적용하는 것을 열림(opening) 연산이라고 하고, 팽창 연산 후 침식 연산을 적용하는 것을 닫힘(closing) 연산이라고 합니다. 열림 연산은 주변보다 밝은 노이즈를 제거하는데 효과적입니다. 또한 맞닿아 있는 것처럼 보이는 독립된 개체를 분리하거나 돌출된 모양을 제거하는 데 효과적입니다. 반면, 닫힘 연산은 주변보다 어두운 노이즈를 제거하는데 효과적이면서 끊어져 보이는 개체를 연결하거나 구멍을 메우는 데 효과적입니다. 

    * 열림 = 침식 + 팽창
    * 닫힘 = 팽창 + 침식

* 팽창 연산을 적용한 이미지에서 침식 연산을 적용한 이미지를 빼면 경계 픽셀만 얻게 되는데, 이는 앞서 살펴본 경계 검출과 비슷합니다. 이런 연산을 그레디언트(gradient) 연산이라고 합니다.

    * 그레디언트 = 팽창 - 침식

* 또한, 원본에서 열림 연산 적용 결과를 빼면 값이 크게 튀는 밝은 영역을 강조할 수 있고, 닫힘 연산 적용 결과에서 원본을 빼면 어두운 부분을 강조할 수 있습니다. 이것을 각각 탑햇(top hat)과 블랙햇(black hat) 연산이라고 합니다.

    * 탑햇 = 원본 - 열림
    * 블랙햇 = 닫힘 - 원본

* dst = cv2.morphologyEx(src, op, kernel, dst, anchor, iteration, borderType, borderValue)
    * src: 입력 영상
    * op: 모폴로지 연산 종류
        * cv2.MORPH_OPEN: 열림 연산
        * cv2.MORPH_COLSE: 닫힘 연산
        * cv2.MORPH_GRADIENT: 그레디언트 연산
        * cv2.MORPH_TOPHAT: 탑햇 연산
        * cv2.MORPH_BLACKHAT: 블랙햇 연산)
    * kernel: 구조화 요소 커널
    * dst(optional): 결과 영상
    * anchor(optional): 커널의 기준점
    * iteration(optional): 연산 반복 횟수
    * borderType(optional): 외곽 영역 보정 방법
    * borderValue(optional): 외곽 영역 보정 값

In [17]:
# 열림과 닫힘 연산으로 노이즈 제거 (morph_open_close.py)

import cv2
import numpy as np

img1 = cv2.imread('img/morph_dot.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('img/morph_hole.png', cv2.IMREAD_GRAYSCALE)    

# 구조화 요소 커널, 사각형 (5x5) 생성 ---①
k = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
# 열림 연산 적용 ---②
opening = cv2.morphologyEx(img1, cv2.MORPH_OPEN, k)
# 닫힘 연산 적용 ---③
closing = cv2.morphologyEx(img2, cv2.MORPH_CLOSE, k)

# 결과 출력
merged1 = np.hstack((img1, opening))
merged2 = np.hstack((img2, closing))
merged3 = np.vstack((merged1, merged2))
cv2.imshow('opening, closing', merged3)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
# 모폴로지 그레이언트 (morph_gradient.py)

import cv2
import numpy as np

img = cv2.imread('img/morphological.png')

# 구조화 요소 커널, 사각형 (3x3) 생성 ---①
k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
# 열림 연산 적용 ---②
gradient = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, k)

# 결과 출력
merged = np.hstack((img, gradient))
cv2.imshow('gradient', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
# 모폴로지 탑햇, 블랙햇 연산 (morph_hat.py)

import cv2
import numpy as np

img = cv2.imread('img/moon_gray.jpg')

# 구조화 요소 커널, 사각형 (5x5) 생성 ---①
k = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
# 탑햇 연산 적용 ---②
tophat = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, k)
# 블랫햇 연산 적용 ---③
blackhat = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, k)

# 결과 출력
merged = np.hstack((img, tophat, blackhat))
cv2.imshow('tophat blackhat', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 6.4 이미지 피라미드
* 이미지 피라미드(image pyramid)란 이미지의 크기를 피라미드처럼 단계적으로 확대하거나 축소하는 작업을 말합니다. 

### 6.4.1 가우시안 피라미드
* 가우시안 필터를 적용한 뒤 이미지 피라미드를 구성하는 것을 가우시안 피라미드(gaussian pyramid)라고 합니다.

* dst = cv2.pyrDown(src, dst, dstsize, borderType)
* dst = cv2.pyrUp(src, dst, dstsize, borderType)
    * src: 입력 영상
    * dst: 결과 영상
    * distsize: 결과 영상 크기
    * borderType: 외곽 보정 방식

In [61]:
# 가우시안 이미지 피라미드 (pyramid_gaussian.py)

import cv2

img = cv2.imread('img/girl.jpg')

# 가우시안 이미지 피라미드 축소
smaller = cv2.pyrDown(img) # img x 1/4
# 가우시안 이미지 피라미드 확대
bigger = cv2.pyrUp(img) # img x 4

# 결과 출력
cv2.imshow('img', img)
cv2.imshow('pyrDown', smaller)
cv2.imshow('pyrUp', bigger)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 6.4.2 라플라시안 피라미드
* cv2.pyrUp() 함수로 이미지를 확대하면 0으로 채워진 행과 열이 새롭게 삽입되므로 원본 이미지보다 화질이 떨어집니다.
* 따라서 cv2.pyrDown() 함수를 적용한 뒤 다시 cv2.pyrUp()을 하면 원본 이미지보다 화질이 많이 떨어집니다.
* 이런 문제점을 개선한 방식이 라플라시안 피라미드(laplacian pyramid)입니다.

In [10]:
# 라플라시안 피라미드로 영상 복원 (pyramid_laplacian.py)

import cv2
import numpy as np

img = cv2.imread('img/taekwonv1.jpg')

# 원본 영상을 가우시안 피라미드로 축소
smaller = cv2.pyrDown(img)
cv2.imshow('img', img)
cv2.imshow('smaller', smaller)

smaller2 = cv2.pyrDown(smaller)
cv2.imshow('smaller', smaller)
cv2.imshow('smaller2', smaller2)

# 축소한 영상을 가우시안 피라미드로 확대
bigger = cv2.pyrUp(smaller2)
cv2.imshow('smaller2', smaller2)
cv2.imshow('bigger', bigger)

bigger2 = cv2.pyrUp(bigger)
cv2.imshow('bigger', bigger)
cv2.imshow('bigger2', bigger2)

cv2.waitKey(0)
cv2.destroyAllWindows()

# 원본에서 확대한 영상 빼기
laplacian = cv2.subtract(img, bigger)
# 확대 한 영상에 라플라시안 영상 더해서 복원
restored = bigger + laplacian

# 결과 출력 (원본 영상, 라플라시안, 확대 영상, 복원 영상)
merged = np.hstack((img, laplacian, bigger, restored))
cv2.imshow('Laplacian Pyramid', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()